<a href="https://colab.research.google.com/github/davidedomini/proximal-policy-optimization-implementation/blob/main/DL_Project_PPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Proximal Policy Optimization (PPO) implementation**

University of Bologna - Department of Computer Science and Engineering

Project for ***Deep Learning*** course

Author: Davide Domini - davide.domini@studio.unibo.it - 0001025049



## **Important remark**

Since there exist several variants of the PPO algorithm, in this project I chose to implement a Clip-based version with pseudocode found in [OpenAI Spinning Up documentation](https://spinningup.openai.com/en/latest/algorithms/ppo.html#id7)

## **Tools**

Several tools were used to implement this project, namely:
- PyTorch
- OpenAI Gym
- OpenCV
- Numpy

## **Preliminary operations**

The following code downloads all the necessary libraries

In [ ]:
!apt-get install -y xvfb ffmpeg > /dev/null 2>&1
!pip install pyvirtualdisplay > /dev/null 2>&1

In [ ]:
!pip install swig > /dev/null 2>&1
!pip install gym[box2d] > /dev/null 2>&1

Defining logs directory

In [ ]:
log_dir = 'logs/rewards'

## **Useful modules import**

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.distributions import MultivariateNormal
from torch.optim import Adam

import numpy as np
import matplotlib.pyplot as plt

import gym

import tensorflow as tf

import cv2
import uuid
from IPython.display import HTML
from base64 import b64encode
from IPython.display import clear_output
import os
from pyvirtualdisplay import Display

## **Utility functions**

The following code defines a function that could be used to create a MP4 video file from a list of frames

In [ ]:
def create_mp4_video_from_frames(frames, fps):
  temp_video_path='tempfile.mp4'
  compressed_path='{}.mp4'.format(str(uuid.uuid4()))

  size=(frames[0].shape[1],frames[0].shape[0])
  out = cv2.VideoWriter(temp_video_path,cv2.VideoWriter_fourcc(*'mp4v'), fps, size)

  for i in range(len(frames)):
      out.write(frames[i][...,::-1].copy())  #rgb[...,::-1].copy()
  out.release()

  os.system(f"ffmpeg -i {temp_video_path} -vcodec libx264 {compressed_path}")

  os.remove(temp_video_path)

  return compressed_path

## **Neural Network Definition**

The following code defines a custom feedforward neural network using PyTorch, the structure is the following:
- Three layers: input, hidden and output
- The input and output size is a parameter passed in from outside
- The hidden size is a hyper-parameter set to 64
- RELU is used as the activation function

In [ ]:
class CustomNN(nn.Module):

  def __init__(self, input_size, output_size):
    super(CustomNN, self).__init__()
    self.input_layer = nn.Linear(input_size, 64)
    self.hidden_layer = nn.Linear(64, 64)
    self.output_layer = nn.Linear(64, output_size)

  def forward(self, obs):
    if isinstance(obs, np.ndarray):
      obs = torch.tensor(obs, dtype=torch.float)

    input_activation = F.relu(self.input_layer(obs))
    hidden_activation = F.relu(self.hidden_layer(input_activation))
    output = F.relu(self.output_layer(hidden_activation))
    return output

## **PPO Algorithm**

<img src="https://raw.githubusercontent.com/davidedomini/proximal-policy-optimization-implementation/main/imgs/PPO-pseudocode.webp" width="800">

### **Important remarks**


#### Actor and critic method


*The Actor-Critic is a combination of value-based, and policy-based methods where the Actor controls how our agent behaves using the Policy gradient, and the Critic evaluates how good the action taken by the Agent based on value-function.*


- The Actor-Critic RL aims to find an optimal policy for the agent in an environment using two components: Actor and Critic;
- Actor: learns an optimal policy by exploring the environment;
- Critic: assesses the value of each action taken by the Actor to determine whether the action will result in a better reward, guiding the Actor for the best course of action to take.

<img src="https://raw.githubusercontent.com/davidedomini/proximal-policy-optimization-implementation/main/imgs/ac_method.webp" width="700">

- The actor takes the current state as input from the environment. Using a neural network, which approximates the policy, it outputs the probabilities of each action for the state;
- The critic takes the current state and the actor's outputted actions as inputs and uses this information to estimate the advantage
  - *Q(s,a)* represents the expected cumulative reward an agent can expect to receive if it follows a certain policy in a given state;
  - *V(s)* represents the expected future reward for a given state, regardless of the action taken;
  - If the advantage function for a particular state-action pair is positive, taking that action in that state is expected to yield a better outcome than the average action taken in that state;
  - The negative value of the advantage function indicates that the current action is less advantageous than expected, and the agent needs to explore other actions or update the policy to improve the performance;
- Finally, the advantage function is backpropagated to both the actor and the critic, allowing both components to continuously update and improve their respective functions.

#### Rewards to go


The reward to go estimates the discounted future reward that can be obtained starting from a given state $S_k$.
It can be calculated as follow (where γ is a parameter in the range$[0, 1]$):



<img src="https://raw.githubusercontent.com/davidedomini/proximal-policy-optimization-implementation/main/imgs/reward-to-go.webp" width="400">

#### Adding exploration

An exploration factor can be added to the actor's choice of action not by directly using the action chosen by the neural network but by using it to go and sample a new one from a probability distribution.

### **Implementation**

In [ ]:
class ProximalPolicyOptimization:

  def __init__(self, environment):

    # Initialize hyperparameters
    self.initialize_hyperparameters()

    # Extract useful envrironment information
    self.environment = environment
    self.observation_size = self.environment.observation_space.shape[0]
    self.action_size = self.environment.action_space.shape[0]

    # ALGORITHM STEP 1: Initialize actor and critic networks
    self.actor = CustomNN(self.observation_size, self.action_size)
    self.critic = CustomNN(self.observation_size, 1)

    # Intialize optimizers
    self.actor_optimizer = Adam(self.actor.parameters(), lr=self.learning_rate)
    self.critic_optimizer = Adam(self.critic.parameters(), lr=self.learning_rate)

    # Initialize distribution for actions selection
    self.cov_var = torch.full(size=(self.action_size,), fill_value=0.5)
    self.cov_matrix = torch.diag(self.cov_var)

    # Initialize tensorboard logger
    self.summary_writer = tf.summary.create_file_writer(log_dir)

    # Initialize all episodic rewards container
    self.all_episodic_rewards = []

  def initialize_hyperparameters(self):
    self.timesteps_per_batch = 4080
    self.max_timesteps_per_episode = 1600
    self.gamma = 0.95
    self.updates_per_iteration = 5
    self.learning_rate = 0.005
    self.clip = 0.2 # Value suggested by the paper

  def get_action(self, observation, testing=False):

    action = self.actor.forward(observation)

    # If we are in training we add an exploratory factor
    if not testing:
      # Create the Multivariate Normal Distribution
      distribution = MultivariateNormal(action, self.cov_matrix)

      # Sample an action from the distribution and get its log probability
      action_with_exploration = distribution.sample()
      log_probability = distribution.log_prob(action_with_exploration)

      return action_with_exploration.detach().numpy(), log_probability.detach()
    else:
      return action.detach().numpy(), 0

  def evaluate_action(self, batch_observations, batch_actions):

    V = self.critic.forward(batch_observations).squeeze()

    mean = self.actor.forward(batch_observations)
    distribution = MultivariateNormal(mean, self.cov_matrix)
    log_probabilities = distribution.log_prob(batch_actions)

    return V, log_probabilities

  def compute_rewards_to_go(self, batch_rewards):

    batch_rtgs = []

    for episode_rewards in reversed(batch_rewards):
      discounted_reward = 0
      for reward in reversed(episode_rewards):
        discounted_reward = reward + discounted_reward * self.gamma
        batch_rtgs.insert(0, discounted_reward)

    batch_rtgs = torch.tensor(batch_rtgs, dtype=torch.float)

    return batch_rtgs

  def log_rewards(self, batch_rewards):
    for episode_rewards in batch_rewards:
      total_reward = np.sum(episode_rewards)
      self.all_episodic_rewards.append(total_reward)

  def rollout(self):

    # Batch data
    batch_observations = []
    batch_actions = []
    batch_log_probabilities = []
    batch_rewards = []
    batch_rewards_to_go = []
    batch_episodes_lengths = []

    time = 0

    while time < self.timesteps_per_batch:
      episode_rewards = []
      observation = self.environment.reset()
      done = False

      for episode_time in range(self.max_timesteps_per_episode):
        time = time + 1
        batch_observations.append(observation)

        action, log_probability = self.get_action(observation)
        observation, reward, done, _ = self.environment.step(action)

        episode_rewards.append(reward)
        batch_actions.append(action)
        batch_log_probabilities.append(log_probability)

        if done:
          break

      batch_episodes_lengths.append(episode_time + 1)
      batch_rewards.append(episode_rewards)

    # Convert data into tensors
    batch_observations = torch.tensor(batch_observations, dtype=torch.float)
    batch_actions = torch.tensor(batch_actions, dtype=torch.float)
    batch_log_probabilities = torch.tensor(batch_log_probabilities, dtype=torch.float)

    # Log episodic rewards
    self.log_rewards(batch_rewards)

    # ALGORITHM STEP 4: Compute rewards to go
    batch_rewards_to_go = self.compute_rewards_to_go(batch_rewards)

    return batch_observations, batch_actions, batch_log_probabilities, batch_rewards_to_go, batch_episodes_lengths

  def learn(self, total_timesteps):
    time = 0
    while time < total_timesteps: # ALGORITHM STEP 2: Iterate through time

      # ALGORITHM STEP 3: Collect set of trajectories
      batch_observations, batch_actions, batch_log_probabilities, batch_rewards_to_go, batch_episodes_lengths = self.rollout()
      time = time + np.sum(batch_episodes_lengths)

      # ALGORITHM STEP 5: Compute advantages
      V, _ = self.evaluate_action(batch_observations, batch_actions) # Compute V_{phi, k}
      A_k = batch_rewards_to_go - V.detach()
      A_k = (A_k - A_k.mean()) / (A_k.std() + 1e-10) # Normalization

      for _ in range(self.updates_per_iteration):
        # Compute V_{phi} and phi_{theta}(a_t | s_t)
        V, current_log_probabilities = self.evaluate_action(batch_observations, batch_actions)

        # ALGORITHM STEP 6: Update actor network

        # Compute ratios
        ratios = torch.exp(current_log_probabilities - batch_log_probabilities)

        # Compute surrogate loss functions
        surrogate_loss_1 = ratios * A_k
        surrogate_loss_2 = torch.clamp(ratios, 1 - self.clip, 1 + self.clip) * A_k

        # Compute actor loss
        actor_loss = (- torch.min(surrogate_loss_1, surrogate_loss_2)).mean()

        # Compute gradients and perform backward propagation for actor network
        self.actor_optimizer.zero_grad()
        actor_loss.backward(retain_graph=True)
        self.actor_optimizer.step()

        # ALGORITHM STEP 7: Update critic network

        # Calculate critic loss
        critic_loss = nn.MSELoss()(V, batch_rewards_to_go)

        # Compute gradients and perform backward propagation for critic network
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step()

  def visualize_single_episode(self):
    frames = []
    done = False
    episode_reward = 0
    state = self.environment.reset()

    while not done:
      frame = self.environment.render(mode='rgb_array')
      frames.append(frame)

      action, _ = self.get_action(state, True)
      state, reward, done, _ = self.environment.step(action)
      episode_reward += reward

    print(f'Total reward: {episode_reward}')

    create_mp4_video_from_frames(frames, 30)

  def save_neural_networks(self):
    torch.save(self.actor.state_dict(), 'actor_nn.pt')
    torch.save(self.critic.state_dict(), 'critic_nn.pt')

  def load_neural_networks(self, actor_path, critic_path):
    self.actor.load_state_dict(torch.load(actor_path))
    self.critic.load_state_dict(torch.load(critic_path))

  def create_reward_chart(self):
    with self.summary_writer.as_default():
      for i, r in enumerate(self.all_episodic_rewards):
        tf.summary.scalar("reward", r, step=i)

## **Algorithm test**

To test the implemented PPO algorithm we will use a Gym environment called *Bipedal walker*

### **Environment description**

This is a simple 4-joint walker robot environment, this notebook uses the *normal* version with slightly uneven terrain.

- Action space: motor speed values in the [-1, 1] range for each of the 4 joints at both hips and knees;
- Observation space: state consists of hull angle speed, angular velocity, horizontal speed, vertical speed, position of joints and joints angular speed, legs contact with ground, and 10 lidar rangefinder measurements. There are no coordinates in the state vector;
- Rewards: reward is given for moving forward, totaling 300+ points up to the far end. If the robot falls, it gets -100. Applying motor torque costs a small amount of points. A more optimal agent will get a better score;
- Starting state: the walker starts standing at the left end of the terrain with the hull horizontal, and both legs in the same position with a slight knee angle;
- Episode termination: the episode will terminate if the hull gets in contact with the ground or if the walker exceeds the right end of the terrain length

<img src="https://raw.githubusercontent.com/davidedomini/proximal-policy-optimization-implementation/main/imgs/bipedal_walker.gif" width=400>

### **Training**

The training should last about 15-20 minutes

In [ ]:
env = gym.make('BipedalWalker-v3')
model = ProximalPolicyOptimization(env)
model.learn(300000)

In [ ]:
model.create_reward_chart()

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs/rewards

In [ ]:
model.visualize_single_episode()

In [ ]:
model.save_neural_networks()

### **Using a pre-trained NN**

In [ ]:
!wget https://raw.githubusercontent.com/davidedomini/proximal-policy-optimization-implementation/main/pre-trained-nn/actor_nn.pt
!wget https://raw.githubusercontent.com/davidedomini/proximal-policy-optimization-implementation/main/pre-trained-nn/critic_nn.pt

In [ ]:
env = gym.make('BipedalWalker-v3')
model = ProximalPolicyOptimization(env)
model.load_neural_networks('actor_nn.pt', 'critic_nn.pt')
model.visualize_single_episode()

The result should be something similar to the gif shown below:

<img src="https://raw.githubusercontent.com/davidedomini/proximal-policy-optimization-implementation/main/imgs/result.gif" width=400>